加载Mnist数据，使用Tensorflow里面内置的加载Mnist数据的方法
这里，我们把Mnist数据直接放在了本文件的同级目录下面，因此也就不需要再去网络上面下载数据了，剩下了网络加载的时间，同时由于google在国内没有办法访问的原因，因此mnist数据有可能会下载不下来

In [122]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, dtype=tf.float32)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


这里我们检查一下数据的维度。我们发现，这里的数据总共分为三类。
+ **mnist.train** 这个数据用于模型的训练
+ **mnist.test** 这里的数据用于模型的测试
+ **mnist.validation** 暂时还不清楚是干什么的

这里。每一个数据的维度都是[*, 784]的，因此我们知道，这里模型的输入特征有784个。

In [2]:
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape, mnist.validation.labels.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


In [3]:
import tensorflow as tf

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 784])
# 这个是实际上真实标签值
y_ = tf.placeholder(tf.float32, [None, 10])

这里的**UbteractiveSesssion()**表示将这个session注册为默认的session，之后的运算会默认的跑在这个session里面。placeholder里面的第二个参数设置输入维度的时候为**None**的作用是不限制条数。

这里我们预备创建一个含单隐层的神经网络，第一层的节点数为100个，第二层的节点数为10个，这十个节点作为最终的输出。
在初始化神经网络参数的时候，对于W参数，我使用的是tf.random_normal来进行初始化，因为为防止各节点在更新值的时候出现节点参数一致的情况。

In [22]:
W1 = tf.Variable(tf.random_normal(shape=[784, 100]))
b1 = tf.Variable(tf.zeros([100]))
W2 = tf.Variable(tf.random_normal(shape=[100, 10]))
b2 = tf.Variable(tf.zeros([10]))

实现神经网络算法：
第一层，我使用了tanh作为第一层的激活函数，同样，我也尝试过ReLU函数，但是使用ReLU函数的最终效果不太理想，甚至是最终的结果会很差，我想这个可能跟ReLU函数的特性有关，ReLU函数会把输入小于零的值归零，但这些小于零的数值可能在这个训练集中的作用要远远重于那些等于零的值或者大于零的值，因此，把它们归零后会妨碍网络的训练。而当我把激活函数改变为tanh函数的时候，神经网络的正确率有了很大的提升。
第二层，因为第二层是作为输出层，因此要使用softmax激活函数，这里要注意的一点是：softmax激活函数是直接作用在**WX+b**上面的，因此，不要再softmax激活函数里面再添加一个激活函数，这样，会让模型训练变得超差。
例如下面我写的这个
```python
    y = tf.nn.softmax(tf.nn.tanh(tf.matmul(a1, W2) + b2))
```
**上面的写法是错误的**因为这已经不是我们普通的softmax函数了，同时，我们应用的损失函数是在正常的softmax的基础上得来的，因此如果是上面的写法，我们下面的损失函数是不适用的，所以使用一个不适用的激活函数训练出来的网络也一定是不适用的。


In [36]:
a1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
y = tf.nn.softmax(tf.matmul(a1, W2) + b2)

设置损失函数
这里的损失函数是使用交叉熵公式，公式如下：
$$H_{y^{'}}(y)=-\sum_{i}y_{i}^{'}log(y_{i})$$

这里首先使用reduce_sum实现上面的公式，这里是对所有输出列的求和，因此，reduction_indices=\[1\],另外，reduce_sum是把所有本次参加训练的样本的损失函数进行求平均。因为我们训练的时候可不是一个样本一个样本的进行训练，而是一个mini-batch一个mini-batch的数据进行训练。所以这里把每一个损失函数的损失相加求平均是为了适用样本集。这个也就是我们经常所说的**成本函数** 

In [77]:
# 这个是计算交叉熵，用来度量损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

这里记一下笔记：之前的时候，我仿照书上的代码使用了梯度下降训练算法，最终无论是训练多少次，最终的正确率大概在 **90%** 之间，有时尽管再进行训练，正确率可能还会 **低于90%** ,因此，我猜测，可能是因为模型对训练集 **过拟合** 了，因此我换了一种方式，采用 **Adam** 算法进行优化，最终发现，正确率能达到 **95%** 左右，*这已经是相当不错了，记得昨天（2018年2月5日，我使用卷积神经网络来训练mnist数据集，而且还训练了将近3个小时，结果也只有96%的正确率）*，因此我可以确信，的确模型之前存在过拟合的情况。这里我们可以看到，采用单隐层神经网络得出的结果要比只有一层的神经网络（书上面的例子）的效果要好很多。

另外，这里的Adam算法的参数我使用了默认的，因为没有什么特殊的要求，我认为默认的参数已经是可以满足我们的需求了吧。这里的学习率我设置为0.01，这是因为如果想让神经网络有较高的正确率，到训练的后期我们也必须使用较小的学习率，这里我的做法是，在刚开始训练的时候我设置为0.1，之后训练一段时间，我设置为0.05，最后感觉差不多了，再改成0.01进行训练。

In [112]:
# 初始化神经网络的训练器
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cross_entropy)

这里，我们调用tf.global_variables_initializer().run()来进行初始化上面的参数。注意：在之后多次训练神经网络的时候不要再调用这条语句了，调用的话就好像那句话“辛辛苦苦几十年，一朝回到解放前。”

另外。我们之前写代码都是这样写的
```python
    init = tf.global_variables_initializer()
    sess.run(init)
```
而这里只是调用了init的run方法就可以初始化，我估计是和上面的那条语句有关（**sess = tf.InteractiveSession()** ）,可能是既然已经把sess作为默认的Session了，因此也就不需要再显示调用sess.run了吧。

In [113]:
tf.global_variables_initializer().run()

这里进行模型的训练，我们首先获得一个mini-batch的训练数据和标签，然后把这个mini-batch的数据喂给我们的神经网络。这里我们循环了10000次。每一个mini-batch-size的大小为100，当然，听说\\(miniBatchSize = 2^{n}\\)会更好。

In [118]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch_xs, y_:batch_ys})

然后就是进行评估了，怎么评估呢，就是看预测的结果和现实是否一致。方法如下：

**tf.argmax()** 是寻找数组中最大的那一个。如果预测的y中最大的位置和实际中y_中最大的位置一样那就是预测正确，否则就是预测失败呗。第二个参数是在哪个维度进行寻找。

因此，最终**tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))** 会返回一个元素为bool值的列表。

In [119]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

这里，通过 **tf.cast()** 将bool值转换为float类型的值，这是为了之后的统计方便，错误的将会被转换为0，正确的将会被转换为1，之后字出来的平均值就是我们模型的正确率。

In [120]:
accutaty = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

最后，我们对模型进行评测，评测调用的函数是**Op.eval()**, 这里我们的**feed_dict={x:mnist.test.images, y_:mnist.test.labels}** 这是因为，我们要用test的数据对模型进行评测。

In [121]:
print(accutaty.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels})*100, "%")

95.99000215530396 %


最终的正确率达到了**95.99%** 证明我们的模型训练的还是挺不错的。